## ESRI World Geocoding Service

The code below sets up calling to the ESRI World Geocoding Service. This geocoder is a very powerful and fast method, able to process 5-10 addresses per second from around the globe. Additionally, it can retrieve both point and extent (rectangle) matches for addresses, which can be helpful when your addresses specify large properties. However, this method requires you to have permissions to create an OAuth 2.0 API Key in ArcGIS Online. The token used below can then be found in the item overview, under 'Credentials'. To learn how to set up an API Key, see the guide to [Creating API Key Credentials](https://developers.arcgis.com/documentation/security-and-authentication/api-key-authentication/api-key-credentials/). 

Developer Credentials are primarily used in application development, so this method should be considered a workaround. Some notes that may be helpful:

 - This method will consume credits, around 1 for every 25 addresses geocoded.
 - The Temporary Token ('token' parameter) will expire in two hours after creation.

In [ ]:
import pandas as pd
df = pd.read_csv('RI_Independent_Schools.csv')

In [ ]:
import requests
# Base URL for the ArcGIS Geocoding API
url = "https://geocode-api.arcgis.com/arcgis/rest/services/World/GeocodeServer/findAddressCandidates"

def geocode_row(row):
    params = {
        "f": "pjson", 
        "singleLine": f"{row['location_address1']},{row['location_city']}",
        "token": "ACCESS_TOKEN" 
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        response_data = response.json()
        top_match = response_data['candidates'][0]
        print(top_match)
        return top_match
    else:
        return None

df['top_match'] = df.apply(geocode_row, axis=1)

In [18]:
df['point'] = df['top_match'].apply(lambda loc: tuple(loc['location'].values()) if loc else None)
df['xmin'] = df['top_match'].apply(lambda loc: loc['extent']['xmin'] if loc else None)
df['ymin'] = df['top_match'].apply(lambda loc: loc['extent']['ymin'] if loc else None)
df['xmax'] = df['top_match'].apply(lambda loc: loc['extent']['xmax'] if loc else None)
df['ymax'] = df['top_match'].apply(lambda loc: loc['extent']['ymax'] if loc else None)

In [19]:
df

,org_ID,parent_ID,code,finance_code,name,name_short_30,name_short_15,org_type_ID,org_type,active,...,role_sort_order,source,OverRideSortOrder,top_match,point,extent,xmin,ymin,xmax,ymax
0,2829,NaN,07353,NaN,A Childs University - Cranston,A Childs University,A Childs Univer,2,School,Y,...,5.0,RIDE_Feb 16 2023 10:41AM,9999,"{'address': '695 Park Ave, Cranston, Rhode Isl...","(-71.42863803144, 41.777885002912)","{'xmin': -71.42963803144, 'ymin': 41.776885002...",-71.429638,41.776885,-71.427638,41.778885
1,2928,NaN,17304,NaN,A Childs University - Smithfield,A Childs University,AChilds Univers,2,School,Y,...,5.0,RIDE_Feb 16 2023 10:41AM,9999,"{'address': 'George Washington Hwy, Smithfield...","(-71.535830763267, 41.907383902586)","{'xmin': -71.536830763267, 'ymin': 41.90638390...",-71.536831,41.906384,-71.534831,41.908384
2,2607,NaN,32340,NaN,Middlebridge School,Middlebridge School,Middlebridge,2,School,Y,...,5.0,RIDE_Feb 16 2023 10:41AM,950,"{'address': '333 Ocean Rd, Narragansett, Rhode...","(-71.458191029475, 41.415620991688)","{'xmin': -71.459191029475, 'ymin': 41.41462099...",-71.459191,41.414621,-71.457191,41.416621
3,3232,NaN,27306,NaN,Sea Rose Montessori Co-op,Sea Rose Montessori Co-op,Sea Rose Montes,2,School,Y,...,NaN,RIDE_Feb 16 2023 10:41AM,9999,"{'address': '324 E Main Rd, Portsmouth, Rhode ...","(-71.262093973934, 41.546744006475)","{'xmin': -71.263093973934, 'ymin': 41.54574400...",-71.263094,41.545744,-71.261094,41.547744
4,3363,42.0,709A1,NaN,Seekonk Christian Academy,Seekonk Christian Academy,Seekonk Christi,2,School,Y,...,NaN,RIDE_Feb 16 2023 10:41AM,9999,"{'address': '95 Sagamore Road, Seekonk, Massac...","(-71.311278143577, 41.804244370109)","{'xmin': -71.312278143577, 'ymin': 41.80324437...",-71.312278,41.803244,-71.310278,41.805244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,1365,NaN,35371,NaN,The Stork's Nest Child Academy III,The Stork's Nest,Stork's Nest,2,School,Y,...,5.0,RIDE_Feb 16 2023 10:41AM,9999,"{'address': '1100 Toll Gate Rd, Warwick, Rhode...","(-71.499452959513, 41.711251010053)","{'xmin': -71.500452959513, 'ymin': 41.71025101...",-71.500453,41.710251,-71.498453,41.712251
63,1438,NaN,38306,NaN,Islamic School of RI,Islamic School of RI,Islamic School,2,School,Y,...,5.0,RIDE_Feb 16 2023 10:41AM,9999,"{'address': '840 Providence St, West Warwick, ...","(-71.492642998459, 41.723518009169)","{'xmin': -71.493642998459, 'ymin': 41.72251800...",-71.493643,41.722518,-71.491643,41.724518
64,1437,NaN,38305,NaN,The Tides School - West Warwick,The Tides School - WW,The Tides,2,School,Y,...,12.0,RIDE_Feb 16 2023 10:41AM,9999,"{'address': '222 Washington St, West Warwick, ...","(-71.527791000683, 41.700965995761)","{'xmin': -71.528791000683, 'ymin': 41.69996599...",-71.528791,41.699966,-71.526791,41.701966
65,1497,NaN,39332,NaN,Hillside Alternative Program,Hillside Alternative Program,Hillside,2,School,Y,...,5.0,RIDE_Feb 16 2023 10:41AM,1300,"{'address': '141 Main St, Woonsocket, Rhode Is...","(-71.514075021965, 42.002406014131)","{'xmin': -71.515075021965, 'ymin': 42.00140601...",-71.515075,42.001406,-71.513075,42.003406


## Geocoding with Geopy
Here we demonstrate how to use geopy, a Python wrapper for calling geocoder APIs and extracting coordinates.

The first case sets up calls to the Nominatim open source geolocator, which is used by OpenStreetMap and other applications to determine address locations. It allows bulk geocoding with an interval of one second between requests. We iterate through our data through a RateLimiter function, which is able to gracefully handle the process of rate limiting API calls. We follow a query format of *address, city* for each call.

In [4]:
import pandas as pd
df = pd.read_csv('RI_Independent_Schools.csv')

In [ ]:
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
geolocator = Nominatim(user_agent="BrownUniversityLibrary")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, max_retries=1)
def geocode_row(row):
    addr = f"{row['location_address1']},{row['location_city']}"
    geocoded_row = geocode(addr)
    print(geocoded_row)
    return geocoded_row

df['location'] = df.apply(geocode_row, axis=1)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

This is able to return coordinates for 64 of 68 locations. 

In [34]:
len(df[df['point'].isna()])

4